In [5]:
import numpy as np
from header import *
from nsp.utils.base_conv import *

### include addfirsting in optimization step

In [3]:
N = 9
X = np.random.randn(N**2, N**2)
X = X.T + X
# mask = np.abs(X)>=(np.max(X)/(N))
# X[mask] = 0
loss_l1 = nsp.loss.MES(torch.from_numpy(X), [N, N])
t = 0.001
ret_min_grad = 1e10
loss_mes = nsp.loss.MES(X, [N, N])
best_fun = 1E10
model = nsp.model.UnitaryRiemanGenerator(N, dtype=torch.float64)
solver = UnitaryTransTs(RiemanUnitaryCG, model, loss_mes, lr = 0.005, momentum=0.1, af = True)
ret = solver.run(10000, False)

Turn on adding first settings. Note that available only for 1D chain
target loss      : 48.9201656388
initial loss     : 110.7061938792
loss upper bound : 110.6870435339




  0%|          | 0/10000 [00:00<?, ?it/s]

/home/project/python/nsp/test/../nsp/optim/rieman_unitary_optim.py:229: UserWarning: An output with one or more elements was resized since it had shape [1, 9, 9], which does not match the required output shape [1, 1, 9, 9]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  return loss(torch.matrix_exp(-t*H)@U).item()
/home/project/python/nsp/test/../nsp/optim/rieman_unitary_optim.py:294: UserWarning: An output with one or more elements was resized since it had shape [1, 9, 9], which does not match the required output shape [1, 1, 9, 9]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resiz

stack in local minimum --> break loop


In [19]:
L = 4
bonds = [[i, (i+1)%L] for i in range(L)]
LH = sum_ham(X, bonds, L, 4)
LH_abs = sum_ham(stoquastic(X), bonds, L, 4)

In [20]:
np.linalg.eigvalsh(LH_abs)[-1], np.linalg.eigvalsh(LH)[-1]

(66.69024444031078, 27.035349234148846)

### added first then stoquastic

In [14]:
L = 4
bonds = [[i, (i+1)%L] for i in range(L)]
LH = sum_ham(X_2, bonds, L, 4)
bonds = [[i, (i+1)%L, (i+2)%L] for i in range(L)]
LH += sum_ham(X_3site,bonds, L, 4)

In [18]:
np.linalg.eigvalsh(LH_abs)[-1], np.linalg.eigvalsh(LH)[-1]

(60.90619596800536, 27.035349234148846)

In [16]:
L = 4
bonds = [[i, (i+1)%L] for i in range(L)]
LH_abs = sum_ham(stoquastic(X_2), bonds, L, 4)
bonds = [[i, (i+1)%L, (i+2)%L] for i in range(L)]
LH_abs += sum_ham(stoquastic(X_3site),bonds, L, 4)

In [17]:
np.linalg.eigvalsh(LH_abs)[-1], np.linalg.eigvalsh(LH)[-1]

(60.90619596800536, 27.035349234148846)

## apply on MG

In [6]:
import numpy as np
import argparse
import sys
sys.path.append('../nsp')
from nsp.utils.func import *


Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


SzSz = np.kron(Sz,Sz).real.astype(np.float64)
SxSx = np.kron(Sx,Sx).real.astype(np.float64)
SySy = np.kron(Sy,Sy).real.astype(np.float64)

lh = SzSz + SxSx + SySy

In [219]:
print("1D chain lattice")
bonds = [[0,1], [0, 2], [1, 2]]
lh2 = sum_ham(lh/2, bonds, 3, 2)
LH = sum_ham(lh2/2, [[0,1,2], [3, 4, 5]], 6, 2) + sum_ham(lh2, [[1, 2, 3], [2, 3, 4]], 6, 2)
# LH = -LH

1D chain lattice


### original

In [220]:
L = 12
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H = sum_ham(LH, bonds, int(L/3), 8)
# print("max_eig : ", np.linalg.eigvalsh(H)[-1])
E_origin = np.linalg.eigvalsh(H)

In [146]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_abs = sum_ham(stoquastic(LH), bonds, int(L/3), 8)
print("max_eig : ", np.linalg.eigvalsh(H_abs)[-1])

max_eig :  6.411629916792666


#### adding first

In [5]:
N = 8
act = [N, N]
I_not1 = torch.logical_not(torch.eye(act[0]))
I_not2 = torch.logical_not(torch.eye(act[1]))
ML = torch.kron(I_not1, torch.eye(act[1]))
MR = torch.kron(torch.eye(act[0]), I_not2)
MI = torch.kron(I_not1, I_not2) + torch.eye(np.prod(act))

LH_3_site = (torch.kron(torch.eye(N),ML * LH) + torch.kron(MR * LH,torch.eye(N)))
LH_2_site = MI * LH

bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_af = sum_ham(LH_2_site, bonds, int(L/3), 8)
bonds = [[i, (i+1)%int(L/3), (i+2)%int(L/3)] for i in range(int(L/3))]
H_af += sum_ham(LH_3_site, bonds, int(L/3), 8)

print("max eig of add first",np.linalg.eigvalsh(H_af)[-1])

bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_af_abs = sum_ham(stoquastic(LH_2_site), bonds, int(L/3), 8)
bonds = [[i, (i+1)%int(L/3), (i+2)%int(L/3)] for i in range(int(L/3))]
H_af_abs += sum_ham(stoquastic(LH_3_site), bonds, int(L/3), 8)
print("max eig of add first (abs)",np.linalg.eigvalsh(H_af_abs)[-1])

NameError: name 'L' is not defined

## optim

In [6]:
X = LH # for N = 8
N = 8
loss_mes = nsp.loss.MES(X, [N, N])
best_fun = 1E10
for _ in range(1):
    model = nsp.model.UnitaryRiemanGenerator(N, dtype=torch.float64)
    solver = UnitaryTransTs(RiemanUnitaryCG, model, loss_mes, lr = 0.005, momentum=0.1)
    ret = solver.run(10000, False)
    if ret.fun < best_fun:
        print(f"\nbest_fun : {ret.fun}\n")
        best_fun = ret.fun
        best_model = model

LH_o = loss_mes._transform([best_model.matrix()]*loss_mes._n_unitaries, original = True).detach().numpy()

target loss      : 2.2500000000
initial loss     : 5.2605534474
dtype changes from torch.float64 to torch.float32
loss upper bound : 2.8182671070




  0%|          | 0/10000 [00:00<?, ?it/s]

dtype changes from torch.float32 to torch.float64


/home/project/python/nsp/test/../nsp/optim/rieman_unitary_optim.py:229: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 8], which does not match the required output shape [1, 1, 8, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  return loss(torch.matrix_exp(-t*H)@U).item()
/home/project/python/nsp/test/../nsp/optim/rieman_unitary_optim.py:294: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 8], which does not match the required output shape [1, 1, 8, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resiz

No local minimum found
5.976805818875855e-13 2.239696041215211e-14 2.692784112571943 2.6927841125719336
No local minimum found
5.900927950665241e-13 2.2396948770619928e-14 2.692784112571943 2.692784112571934
stack in local minimum --> break loop

best_fun : 2.6927841125719425



In [10]:
L = 12

In [11]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_o = sum_ham(LH_o, bonds, int(L/3), 8)
print("max eig",np.linalg.eigvalsh(H_o)[-1])

max eig 4.499999999999559


In [12]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_o_abs = sum_ham(stoquastic(LH_o), bonds, int(L/3), 8)
print("max eig",np.linalg.eigvalsh(H_o_abs)[-1])

max eig 5.879777583231808


### add_first

In [39]:
act = loss_mes.act.tolist()
I_not1 = torch.logical_not(torch.eye(act[0]))
I_not2 = torch.logical_not(torch.eye(act[1]))
ML = torch.kron(I_not1, torch.eye(act[1]))
MR = torch.kron(torch.eye(act[0]), I_not2)
MI = torch.kron(I_not1, I_not2) + torch.eye(np.prod(act))

In [40]:
LH_3_site = (torch.kron(torch.eye(N),ML * LH_o) + torch.kron(MR * LH_o,torch.eye(N)))
LH_2_site = MI * LH_o

In [41]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_af = sum_ham(LH_2_site, bonds, int(L/3), 8)
bonds = [[i, (i+1)%int(L/3), (i+2)%int(L/3)] for i in range(int(L/3))]
H_af += sum_ham(LH_3_site, bonds, int(L/3), 8)

In [42]:
E = np.linalg.eigvalsh(H_af)

In [45]:
np.sum(E*np.exp(E))/np.exp(E).sum()

2.2635109996382816

In [153]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_af_abs = sum_ham(stoquastic(LH_2_site), bonds, int(L/3), 8)
bonds = [[i, (i+1)%int(L/3), (i+2)%int(L/3)] for i in range(int(L/3))]
H_af_abs += sum_ham(stoquastic(LH_3_site), bonds, int(L/3), 8)


In [154]:
print("max eig of add first (abs)",np.linalg.eigvalsh(H_af_abs)[-1])

max eig of add first (abs) 5.90262796484217


### test multi 

In [68]:
folder1 = "/home/project/python/array/majumdar_ghosh/optimize8_2"
LHs = [np.load(folder1 + f"/{i}.npy") for i in range(2)]

In [69]:
bonds = [[i, (i+1)%int(L/3)] for i in range(int(L/3))]
H_o = sum_ham(LHs[0], bonds[::2], int(L/3), 8) + sum_ham(LHs[1], bonds[1::2], int(L/3), 8) 
E = np.linalg.eigvalsh(H_o)
print("max eig",E[-1])

max eig 4.4999999999951905


#### adding

In [70]:
folder1 = "/home/project/python/array/majumdar_ghosh/optimize8_2_af"
LHs = [np.load(folder1 + f"/{i}.npy") for i in range(4)]

In [13]:
from random import randint

L = 4
M = 1
Sz = np.zeros([2,2])
Sz[0,0] = 1/2
Sz[1,1] = -1/2
Sx = np.zeros([2,2])
Sx[1,0] = 1/2
Sx[0,1] = 1/2
Sy = np.zeros([2,2], dtype=np.complex64)
Sy[1,0] = 1j/2
Sy[0,1] = -1j/2


SzSz = np.kron(Sz,Sz).real.astype(np.float64)
SxSx = np.kron(Sx,Sx).real.astype(np.float64)
SySy = np.kron(Sy,Sy).real.astype(np.float64)

lh = SzSz + SxSx + SySy

lh = -lh # use minus of local hamiltonian for monte-carlo (exp(-beta H ))
bonds = [[0,1], [0, 2], [1, 2]]
lh2 = sum_ham(lh/2, bonds, 3, 2)
LH = sum_ham(lh2/2, [[0,1,2], [3, 4, 5]], 6, 2) + sum_ham(lh2, [[1, 2, 3], [2, 3, 4]], 6, 2)

D = 8
models = [nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64) for _ in range(L)]
model = copy.deepcopy(models[0])
loss = nsp.loss.MES(LH, [D,D])


# set_seed(33244233)
# X = LH
t = 0.001
ret_min_grad = 1e10

best_fun = 1E10
print(f"iteration : {M}")
for _ in range(M):
    seed = randint(0, 2<<32 - 1)
    torch.manual_seed(seed)
    np.random.seed(seed)
    models = [nsp.model.UnitaryRiemanGenerator(D, dtype=torch.float64) for _ in range(L)]
    cg2 = RiemanNonTransUnitarySGD([(models[i], models[(i+1)%L]) for i in range(L)], [loss]*L, pout = False, lr = 0.01, momentum = 0.05)
    # cg2 = RiemanNonTransUnitaryCG([(models[i], models[(i+1)%L]) for i in range(L)], [loss]*L, pout = False)
    solver2 = UnitaryNonTransTs(cg2, af = True)
    ret = solver2.run(2000, disable_message=False)
    print(f"res = {ret.fun} / seed = {seed}")
    if ret.fun < best_fun:
        print(f"\nbest_fun updated : {ret.fun}\n")
        best_fun = ret.fun
        best_model = ret.best_model

iteration : 1
Turn on adding first settings. Note that available only for 1D chain
target loss      : 9.0000000000
initial loss     : 20.5325985331
loss upper bound : 11.0849433715




  0%|          | 0/2000 [00:00<?, ?it/s]

/home/project/python/nsp/test/../nsp/optim/rieman_nontrans_unitary_optim.py:255: UserWarning: An output with one or more elements was resized since it had shape [1, 8, 8], which does not match the required output shape [1, 1, 8, 8]. This behavior is deprecated, and in a future PyTorch release outputs will not be resized unless they have zero elements. You can explicitly reuse an out tensor t by resizing it, inplace, to zero elements with t.resize_(0). (Triggered internally at  ../aten/src/ATen/native/Resize.cpp:17.)
  param.data = (torch.matrix_exp(inv_step_dir * -lr) @ U).view(-1)


KeyboardInterrupt: 

In [200]:
np.linalg.eigvalsh(LH)

array([-1.125     , -1.125     , -1.125     , -1.125     , -1.125     ,
       -1.125     , -1.125     , -0.91929537, -0.91929537, -0.91929537,
       -0.91929537, -0.91929537, -0.75      , -0.75      , -0.75      ,
       -0.75      , -0.75      , -0.61622747, -0.61622747, -0.61622747,
       -0.35395916, -0.35395916, -0.35395916, -0.35395916, -0.35395916,
       -0.20900301, -0.20900301, -0.20900301, -0.16316027, -0.16316027,
       -0.16316027, -0.125     ,  0.25      ,  0.25      ,  0.25      ,
        0.25      ,  0.25      ,  0.36183442,  0.36183442,  0.36183442,
        0.43379923,  0.43379923,  0.43379923,  0.625     ,  0.625     ,
        0.625     ,  0.625     ,  0.625     ,  0.64825453,  0.64825453,
        0.64825453,  0.64825453,  0.64825453,  0.89143125,  0.89143125,
        0.89143125,  0.92632585,  0.92632585,  0.92632585,  1.125     ,
        1.125     ,  1.125     ,  1.125     ,  1.125     ])

In [125]:
best_model = ret.model

In [37]:
L = 4
LHs = [loss._transform_kron([best_model[i].matrix(), best_model[(i+1)%L].matrix()]).detach().numpy() for i in range(L)]

In [38]:
L_ = 4
H = sum_ham(LHs[0], [[0,1]], L_, 8)
H += sum_ham(LHs[1], [[1,2]], L_, 8)
H += sum_ham(LHs[2], [[2,3]], L_, 8)
H += sum_ham(LHs[3], [[3,0]], L_, 8)

In [49]:
E_4 = np.linalg.eigvalsh(H)
E_4_iaf = np.linalg.eigvalsh(stoquastic(H))

In [45]:
L_ = 4
H_4_af = sum_ham(stoquastic(LHs[0]), [[0,1]], L_, 8)
H_4_af += sum_ham(stoquastic(LHs[1]), [[1,2]], L_, 8)
H_4_af += sum_ham(stoquastic(LHs[2]), [[2,3]], L_, 8)
H_4_af += sum_ham(stoquastic(LHs[3]), [[3,0]], L_, 8)

In [46]:
E_4_af = np.linalg.eigvalsh(H_4_af)

In [126]:
models = best_model
X = []
for acts, loss in zip(cg2.models_act, cg2.loss_list):
    X.append(loss._preprocess([models[a].matrix() for a in acts]))

In [82]:
L_ = 4
H = sum_ham(X[0].detach().numpy(), [[0,1]], L_, 8)
H += sum_ham(X[1].detach().numpy(), [[1,2]], L_, 8)
H += sum_ham(X[2].detach().numpy(), [[2,3]], L_, 8)
H += sum_ham(X[3].detach().numpy(), [[3,0]], L_, 8)

E_prime = np.linalg.eigvalsh(H)

In [127]:
D = 8
I_not = torch.logical_not(torch.eye(D))
I = torch.eye(D)
MI = torch.kron(I_not, I_not) + torch.eye(D**2)   
ML = torch.kron(I_not, torch.eye(D)) + MI/2
MR = torch.kron(torch.eye(D), I_not) + MI/2

In [128]:
X_ = [(torch.kron(I,ML * X[(i+1)%len(X)]) + torch.kron(MR * X[i],I)) for i in range(len(X))]
X_ = [x.detach().numpy() for x in X_]

In [131]:
loss1

10.353602142883915

In [130]:
loss1 = 0
for x in X_:
    loss1 += loss.forward(x)

In [122]:
cg2.loss_val()

tensor([10.6059], dtype=torch.float64, grad_fn=<AddBackward0>)

In [99]:
L_ = 4
H_3 = sum_ham(stoquastic(X_[0]), [[0,1,2]], L_, 8)
H_3 += sum_ham(stoquastic(X_[1]), [[1,2,3]], L_, 8)
H_3 += sum_ham(stoquastic(X_[2]), [[2,3,0]], L_, 8)
H_3 += sum_ham(stoquastic(X_[3]), [[3,0,1]], L_, 8)

In [100]:
E_3_iaf = np.linalg.eigvalsh(stoquastic(H_3))

In [115]:
best_model

[UnitaryRiemanGenerator(),
 UnitaryRiemanGenerator(),
 UnitaryRiemanGenerator(),
 UnitaryRiemanGenerator()]

In [113]:
cg2 = RiemanNonTransUnitarySGD([(best_model[i], best_model[(i+1)%L]) for i in range(L)], [loss]*L, pout = False, lr = 0.01, momentum = 0.05)

In [64]:
name = [0, 1, 2, 3]
hams = [np.load(f"/home/project/python/array/AKLT/optm2_nt2_mes_J=2.0/{n}.npy") for n in name]

In [65]:
hams2 = hams[:z]
hams3 = hams[4:]
co = nsp.utils.base_conv.change_order
LH_3 = [co(lh, [8, 8, 8]) for lh in hams3]
LH_2 = [co(lh, [8, 8]) + np.eye(lh.shape[0])*1.125 for lh in hams2]

In [66]:
LH_33  = []
for i in range(4):
    LH_33.append(LH_3[i] + np.kron(LH_2[i]/2, np.eye(8)) + np.kron(np.eye(8), LH_2[(i+1)%4]/2))

In [67]:
loss1 = 0
for x in LH_33:
    loss1 += loss.forward(x)

In [68]:
loss1 

19.205776321845327

In [186]:
L_ = 4
H_33 = sum_ham(stoquastic(LH_33[0]), [[0,1,2]], L_, 8)
H_33 += sum_ham(stoquastic(LH_33[1]), [[1,2,3]], L_, 8)
H_33 += sum_ham(stoquastic(LH_33[2]), [[2,3,0]], L_, 8)
H_33 += sum_ham(stoquastic(LH_33[3]), [[3,0,1]], L_, 8)

In [187]:
E_333 = np.linalg.eigvalsh(H_33)

In [189]:
(E_333 + 4.5)[-10:]

array([9.03278129, 9.04443296, 9.04447931, 9.11380781, 9.11382378,
       9.29890728, 9.43219942, 9.65902022, 9.65910413, 9.91717051])

In [159]:
L_ = 4
H_3 = sum_ham(stoquastic(LH_3[0]), [[0,1,2]], L_, 8)
H_3 += sum_ham(stoquastic(LH_3[1]), [[1,2,3]], L_, 8)
H_3 += sum_ham(stoquastic(LH_3[2]), [[2,3,0]], L_, 8)
H_3 += sum_ham(stoquastic(LH_3[3]), [[3,0,1]], L_, 8)

H_3 += sum_ham(stoquastic(LH_2[0]), [[0,1]], L_, 8)
H_3 += sum_ham(stoquastic(LH_2[1]), [[1,2]], L_, 8)
H_3 += sum_ham(stoquastic(LH_2[2]), [[2,3]], L_, 8)
H_3 += sum_ham(stoquastic(LH_2[3]), [[3,0]], L_, 8)

In [160]:
E_33 = np.linalg.eigvalsh(H_3)

In [161]:
(E_33 + 4.5)[-50:]

array([8.23841571, 8.24641235, 8.25374496, 8.25799583, 8.25804177,
       8.27124649, 8.2712883 , 8.27287063, 8.29687878, 8.29711798,
       8.29715053, 8.33266612, 8.37969671, 8.38953372, 8.3895816 ,
       8.42701942, 8.42702271, 8.44694611, 8.44695707, 8.48527957,
       8.54231721, 8.54692224, 8.56585219, 8.56885378, 8.5765232 ,
       8.59168667, 8.59172993, 8.69817799, 8.69820751, 8.80098724,
       8.80103555, 8.85056855, 8.85068105, 8.88372041, 8.91060205,
       8.9430519 , 8.98112921, 8.98910652, 8.99033886, 8.99078521,
       9.03278129, 9.04443296, 9.04447931, 9.11380781, 9.11382378,
       9.29890728, 9.43219942, 9.65902022, 9.65910413, 9.91717051])

In [230]:
np.exp(4*E_4_iaf).sum() / np.exp(-4*(E_origin-4.5)).sum()

22.877618264689552

In [229]:
np.exp(4*(E_33 + 4.5)).sum() / np.exp(-4*(E_origin-4.5)).sum()

13.945229661781077

In [29]:
hams = [np.load(f"/home/project/python/array/majumdar_ghosh/optimize8_af/{n}.npy") for n in np.arange(2)]

In [8]:
hams[1] = co(hams[1], [8, 8, 8])
hams[0] = co(hams[0], [8, 8])

NameError: name 'co' is not defined

In [4]:
L_ = 4
H_8_af = sum_ham(hams[1], [[0,1,2]], L_, 8)
H_8_af += sum_ham(hams[1], [[1,2,3]], L_, 8)
H_8_af += sum_ham(hams[1], [[2,3,0]], L_, 8)
H_8_af += sum_ham(hams[1], [[3,0,1]], L_, 8)

H_8_af += sum_ham(hams[0], [[0,1]], L_, 8)
H_8_af += sum_ham(hams[0], [[1,2]], L_, 8)
H_8_af += sum_ham(hams[0], [[2,3]], L_, 8)
H_8_af += sum_ham(hams[0], [[3,0]], L_, 8)

In [171]:
E_8 = np.linalg.eigvalsh(H_8_af)

In [172]:
E_8 + 4.5

array([3.68594044e-13, 6.98108238e-13, 9.38804590e-13, ...,
       8.68086623e+00, 9.00000000e+00, 9.00000000e+00])

In [211]:
E_origin

array([-4.5       , -4.5       , -4.5       , ...,  4.18086623,
        4.5       ,  4.5       ])

In [221]:
(E_origin * np.exp(-1 * E_origin)).sum() /  np.exp(-1 * E_origin).sum()

-2.263510999638674

In [217]:
E_origin

array([-4.5       , -4.5       , -4.5       , ...,  4.18086623,
        4.5       ,  4.5       ])

<function nsp.utils.func.l1_measure(X)>

In [54]:
N = 9
X = np.random.randn(N**2, N**2)
X = X.T + X

In [55]:
nsp.utils.func.l1_measure(X)

7718.742156090741

In [62]:
((X - stoquastic(X))/2).sum()

-3859.3710780453707

In [57]:
X

array([[-2.02237703,  3.77630275, -0.71301597, ..., -2.5382281 ,
        -1.26538107, -2.51056333],
       [ 3.77630275, -1.24366319, -3.87237618, ...,  0.08905426,
         0.5042383 ,  0.50323521],
       [-0.71301597, -3.87237618,  0.39587544, ...,  1.08899415,
         1.81309638,  0.77205963],
       ...,
       [-2.5382281 ,  0.08905426,  1.08899415, ...,  3.25962257,
        -0.73955418,  1.34674103],
       [-1.26538107,  0.5042383 ,  1.81309638, ..., -0.73955418,
        -0.1730198 ,  1.11119799],
       [-2.51056333,  0.50323521,  0.77205963, ...,  1.34674103,
         1.11119799,  2.18836317]])